In [1]:
import pandas as pd
import glob 
import os


In [2]:
df1=pd.read_csv('products.csv')
df1.head(10)

,product_id,product_description,product_category
0,P01,detergent,house
1,P02,kitchen roll,house
2,P03,bin liners,house
3,P04,shower gel,house
4,P05,scented candles,house
5,P06,fabric softener,house
6,P07,cling film,house
7,P08,aluminium foil,house
8,P09,toilet paper,house
9,P10,kitchen knife,house


In [3]:
df2=pd.read_csv('customers.csv')
df2.head(10)

,customer_id,loyalty_score
0,C1,7
1,C2,4
2,C3,8
3,C4,5
4,C5,7
5,C6,10
6,C7,3
7,C8,7
8,C9,8
9,C10,5


In [4]:
txnl=glob.glob('transactions/*/*.json')

In [5]:
txn=pd.DataFrame()
for i in txnl:
    txn=pd.concat([txn,pd.read_json(i,lines=True)],ignore_index=True,axis=0)

In [6]:
txn=txn.drop('date_of_purchase',axis=1)

In [7]:
txn.groupby('customer_id').agg(list)

,basket
customer_id,
C1,"[[{'product_id': 'P38', 'price': 598}, {'produ..."
C10,"[[{'product_id': 'P38', 'price': 970}], [{'pro..."
C100,"[[{'product_id': 'P40', 'price': 1767}, {'prod..."
C101,"[[{'product_id': 'P27', 'price': 1369}, {'prod..."
C102,"[[{'product_id': 'P22', 'price': 315}], [{'pro..."
...,...
C95,"[[{'product_id': 'P34', 'price': 1530}], [{'pr..."
C96,"[[{'product_id': 'P34', 'price': 1101}, {'prod..."
C97,"[[{'product_id': 'P09', 'price': 965}, {'produ..."


In [8]:
max(txn['basket'].apply(len))

3

In [9]:
txn.head()

,customer_id,basket
0,C4,"[{'product_id': 'P37', 'price': 323}, {'produc..."
1,C10,"[{'product_id': 'P38', 'price': 970}]"
2,C14,"[{'product_id': 'P64', 'price': 586}, {'produc..."
3,C17,"[{'product_id': 'P64', 'price': 882}, {'produc..."
4,C22,"[{'product_id': 'P26', 'price': 918}, {'produc..."


In [10]:
res=pd.DataFrame(txn['basket'].tolist(),txn['customer_id'].tolist())

In [11]:
res

,0,1,2
C4,"{'product_id': 'P37', 'price': 323}","{'product_id': 'P39', 'price': 769}",None
C10,"{'product_id': 'P38', 'price': 970}",None,None
C14,"{'product_id': 'P64', 'price': 586}","{'product_id': 'P62', 'price': 48}","{'product_id': 'P61', 'price': 609}"
C17,"{'product_id': 'P64', 'price': 882}","{'product_id': 'P63', 'price': 771}",None
C22,"{'product_id': 'P26', 'price': 918}","{'product_id': 'P32', 'price': 866}","{'product_id': 'P27', 'price': 911}"
...,...,...,...
C117,"{'product_id': 'P32', 'price': 1769}","{'product_id': 'P30', 'price': 1090}","{'product_id': 'P28', 'price': 718}"
C123,"{'product_id': 'P20', 'price': 444}","{'product_id': 'P18', 'price': 1638}","{'product_id': 'P13', 'price': 651}"
C125,"{'product_id': 'P31', 'price': 1807}","{'product_id': 'P22', 'price': 1647}",None
C129,"{'product_id': 'P62', 'price': 594}","{'product_id': 'P62', 'price': 1131}",None


In [12]:
for i in range(len(res.columns)):
    res[['new_prod'+str(i),'new_price'+str(i)]]=res[i].apply(pd.Series)

In [13]:
res=res[['new_prod0','new_prod1','new_prod2']]

In [14]:
res1=res.stack().reset_index()

In [15]:
res1.head()

,level_0,level_1,0
0,C4,new_prod0,P37
1,C4,new_prod1,P39
2,C10,new_prod0,P38
3,C14,new_prod0,P64
4,C14,new_prod1,P62


In [16]:
res1=res1.drop('level_1',axis=1)

In [17]:
res1

,level_0,0
0,C4,P37
1,C4,P39
2,C10,P38
3,C14,P64
4,C14,P62
...,...,...
4336,C125,P31
4337,C125,P22
4338,C129,P62
4339,C129,P62


In [18]:
res1.columns='customer_id','product_id'

In [19]:
res1

,customer_id,product_id
0,C4,P37
1,C4,P39
2,C10,P38
3,C14,P64
4,C14,P62
...,...,...
4336,C125,P31
4337,C125,P22
4338,C129,P62
4339,C129,P62


In [20]:
res1['purchase_count']=1

In [21]:
final=res1.groupby(['customer_id','product_id']).sum().reset_index()

In [22]:
final.head()

,customer_id,product_id,purchase_count
0,C1,P03,1
1,C1,P07,1
2,C1,P36,1
3,C1,P37,4
4,C1,P38,2


In [23]:
final=final.merge(df1,how='left',on='product_id').drop('product_description',axis=1)

In [24]:
final=final.merge(df2,how='left',on='customer_id')

In [28]:
final=final[['customer_id','loyalty_score','product_id','product_category','purchase_count']]
final

,customer_id,loyalty_score,product_id,product_category,purchase_count
0,C1,7,P03,house,1
1,C1,7,P07,house,1
2,C1,7,P36,sweets,1
3,C1,7,P37,sweets,4
4,C1,7,P38,sweets,2
...,...,...,...,...,...
2145,C99,9,P47,food,2
2146,C99,9,P52,food,2
2147,C99,9,P53,food,1
2148,C99,9,P59,food,2


In [26]:
final.to_json(path_or_buf = 'output.json', orient='index')